In [ ]:
import pandas as pd

In [ ]:
#keeping long/lat as string
df = pd.read_csv('data/DonneesOuvertes.csv', dtype=str)
df.info()

# Create the station list
- a reference dataframe
- cleaning dupe due to different spelling of station names based on lat/lon
- fixing missing information. 
- reduce the size of the main dataframe

In [ ]:
start_stations = df[['STARTSTATIONNAME','STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE','STARTSTATIONARRONDISSEMENT']]
end_stations = df[['ENDSTATIONNAME','ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE','ENDSTATIONARRONDISSEMENT']]
start_stations.columns = ['name','latitude','longitude','arrondissement']
end_stations.columns = ['name','latitude','longitude','arrondissement']
stations = pd.concat([start_stations, end_stations]).drop_duplicates().dropna().reset_index(drop=True)

In [ ]:
# some stations have different names
stations[stations[['latitude','longitude']].duplicated(keep=False)].sort_values('latitude')

In [ ]:
to_drop = [839,551,758,173,931,914,926,919,728,923, 652, 705, 780, 787, 48, 893, 927, 858, 381, 376]
stations = stations.drop(to_drop).reset_index(drop=True)
stations.index = stations.index + 1
stations[['latitude','longitude']].duplicated().sum()

In [ ]:
stations.info()

In [ ]:
stations[stations['latitude'] == '-1']

In [ ]:
# got station coordinate from another set
stations.at[358,'latitude'] = '45.492837758845226'
stations.at[358,'longitude'] = '-73.5564199090004'
stations[stations['latitude'] == '-1']

In [ ]:
stations.index.rename('station_id', inplace=True)

In [ ]:
#stations.to_csv('stations_2023.csv')
stations.to_feather('stations_2023.feather')

In [ ]:
station_keys = stations[['latitude','longitude']].reset_index()

In [ ]:
# add the station key in the df then drop cols that are not needed
merge_df = df.merge(station_keys.add_prefix('start_'), left_on=['STARTSTATIONLATITUDE','STARTSTATIONLONGITUDE'], right_on=['start_latitude','start_longitude'], how='left')
merge_df = merge_df.merge(station_keys.add_prefix('end_'), left_on=['ENDSTATIONLATITUDE','ENDSTATIONLONGITUDE'], right_on=['end_latitude','end_longitude'], how='left')
cols_to_drop = ['STARTSTATIONNAME', 'STARTSTATIONARRONDISSEMENT',
       'STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE', 'ENDSTATIONNAME',
       'ENDSTATIONARRONDISSEMENT', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE',
       'start_latitude', 'start_longitude',
       'end_latitude', 'end_longitude',]
merge_df = merge_df.drop(cols_to_drop, axis=1)

In [ ]:
merge_df.head()

In [ ]:
# some stations are not recorded rename as -1
print(merge_df.isna().sum())
merge_df['start_station_id'] = merge_df['start_station_id'].fillna(-1).astype('int16')
merge_df['end_station_id'] = merge_df['end_station_id'].fillna(-1).astype('int16')
print(merge_df.isna().sum())

In [ ]:
merge_df['STARTTIMEMS'] = merge_df['STARTTIMEMS'].astype(float)
merge_df['ENDTIMEMS'] = merge_df['ENDTIMEMS'].astype(float)
merge_df['duration_s'] = round((merge_df['ENDTIMEMS'] - merge_df['STARTTIMEMS']) / 1000)
merge_df['start_timestamp'] = pd.to_datetime(merge_df['STARTTIMEMS'], unit='ms')
merge_df['end_timestamp'] = pd.to_datetime(merge_df['ENDTIMEMS'], unit='ms')
merge_df.drop(['STARTTIMEMS','ENDTIMEMS'], inplace=True, axis=1)

In [ ]:
print(merge_df.info())
print(merge_df.head())
print(merge_df.isna().sum())

In [ ]:
#merge_df.to_csv('n_usage_2023.csv')
merge_df.to_feather('bixi_usage_2023.feather')